In [59]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from keras.src.utils.dataset_utils import labels_to_dataset
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.regularizers import l2


In [83]:
file = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
file.head()
x=file.drop(["id","NObeyesdad"],axis=1)
y=file["NObeyesdad"]
label_encoder = LabelEncoder()
scaler = StandardScaler()


KeyError: "['NObeyesdad'] not found in axis"

In [61]:
y_encoded_train = label_encoder.fit_transform(y)
columns_to_encode = ['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS']

for features in columns_to_encode:
    x[features]=label_encoder.fit_transform(x[features])
x.head()
# print(y_encoded_train)
# print(x_encoded_train)
# file.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,1,24.443011,1.699998,81.669950,1,1,2.000000,2.983297,2,0,2.763573,0,0.000000,0.976473,1,3
1,0,18.000000,1.560000,57.000000,1,1,2.000000,3.000000,1,0,2.000000,0,1.000000,1.000000,2,0
2,0,18.000000,1.711460,50.165754,1,1,1.880534,1.411685,2,0,1.910378,0,0.866045,1.673584,2,3
3,0,20.952737,1.710730,131.274851,1,1,3.000000,3.000000,2,0,1.674061,0,1.467863,0.780199,1,3
4,1,31.641081,1.914186,93.798055,1,1,2.679664,1.971472,2,0,1.979848,0,1.967973,0.931721,1,3


In [62]:
x_encoded_normalized_train=scaler.fit_transform(x)
x_normalized_df = pd.DataFrame(x_encoded_normalized_train, columns=x.columns)
print(x_normalized_df.head())

     Gender       Age    Height    Weight  family_history_with_overweight  \
0  1.004152  0.105699 -0.002828 -0.235713                        0.469099   
1 -0.995866 -1.027052 -1.606291 -1.170931                        0.469099   
2 -0.995866 -1.027052  0.128451 -1.430012                        0.469099   
3 -0.995866 -0.507929  0.120090  1.644770                        0.469099   
4  1.004152  1.371197  2.450367  0.224054                        0.469099   

      FAVC      FCVC       NCP      CAEC     SMOKE      CH2O       SCC  \
0  0.30588 -0.836279  0.314684  0.337845 -0.109287  1.206594 -0.185009   
1  0.30588 -0.836279  0.338364 -1.889204 -0.109287 -0.048349 -0.185009   
2  0.30588 -1.060332 -1.913423  0.337845 -0.109287 -0.195644 -0.185009   
3  0.30588  1.039171  0.338364  0.337845 -0.109287 -0.584035 -0.185009   
4  0.30588  0.438397 -1.119801  0.337845 -0.109287 -0.081469 -0.185009   

        FAF       TUE      CALC    MTRANS  
0 -1.171141  0.597438 -0.471288  0.429319  
1  0

In [92]:
model=keras.Sequential([
    layers.Input(shape=(x_encoded_normalized_train.shape[1],)),
    layers.Dense(256, activation='relu', kernel_regularizer=l2(0.001)),  # Input layer and first hidden layer
    layers.Dense(256, activation='relu', kernel_regularizer=l2(0.001)), 
    layers.Dense(128, activation='relu', kernel_regularizer=l2(0.001)),  
    layers.Dense(16, activation='relu', kernel_regularizer=l2(0.001)),  
    layers.Dense(7, activation='softmax')]
)
early_stopping = EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history = model.fit(x_encoded_normalized_train, y_encoded_train, 
                    epochs=10000, batch_size=16, 
                    validation_split=0.2,  # 20% of the data for validation
                    callbacks=[early_stopping])
train_loss, train_accuracy = model.evaluate(x_encoded_normalized_train, y_encoded_train)


Epoch 1/10000
1038/1038 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5797 - loss: 1.7448 - val_accuracy: 0.8035 - val_loss: 0.9679
Epoch 2/10000
1038/1038 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8277 - loss: 0.9053 - val_accuracy: 0.8372 - val_loss: 0.8249
Epoch 3/10000
1038/1038 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8497 - loss: 0.7968 - val_accuracy: 0.8521 - val_loss: 0.7656
Epoch 4/10000
1038/1038 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8607 - loss: 0.7444 - val_accuracy: 0.8574 - val_loss: 0.7346
Epoch 5/10000
1038/1038 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8679 - loss: 0.7070 - val_accuracy: 0.8630 - val_loss: 0.6930
Epoch 6/10000
1038/1038 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8683 - loss: 0.6744 - val_accuracy: 0.8673 - val_loss: 0.6723
Epoch 7/10000
1038/1038 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8775 - loss: 0.6447 - val_accuracy: 0.8671 - val_loss: 0.6516
Epoch 8/10000
1038/1038 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

<Figure size 800x600 with 0 Axes>